<a href="https://colab.research.google.com/github/jaothan26/digital_assets_analysis_/blob/main/OK_Copy_of_241109_01_BitcoinNetworkRatios_uniquement_code_avec_une_seule_fonction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import chart_utils

In [ ]:
'''#https://github.com/coinmetrics/data/blob/master/csv/btc.csv
data = pd.read_csv('/content/btc.csv',
                   usecols=['time', 'CapMrktCurUSD', 'CapRealUSD', 'TxTfrValAdjUSD', 'PriceUSD',
                            'AdrActCnt', 'CapMVRVCur', 'NVTAdj','VelCur1yr', 'TxCnt', 'SplyCur', 'HashRate']).dropna()  #time for date'''

"#https://github.com/coinmetrics/data/blob/master/csv/btc.csv\ndata = pd.read_csv('/content/btc.csv',\n                   usecols=['time', 'CapMrktCurUSD', 'CapRealUSD', 'TxTfrValAdjUSD', 'PriceUSD',\n                            'AdrActCnt', 'CapMVRVCur', 'NVTAdj','VelCur1yr', 'TxCnt', 'SplyCur', 'HashRate']).dropna()  #time for date"

In [ ]:
#data.dtypes

In [ ]:
'''#https://github.com/coinmetrics/data/blob/master/csv/btc.csv
data2 = pd.read_csv('/content/btc.csv').dropna()  #time for date'''

"#https://github.com/coinmetrics/data/blob/master/csv/btc.csv\ndata2 = pd.read_csv('/content/btc.csv').dropna()  #time for date"

In [ ]:
#https://pypi.org/project/coinmetrics-api-client/
#https://coinmetrics.github.io/api-client-python/site/api_client.html
!pip install coinmetrics.api_client --quiet
from coinmetrics.api_client import CoinMetricsClient
client = CoinMetricsClient()
print(client)


#value time not supported but integrated in the DF
metrics = client.get_asset_metrics(assets = 'btc' ,
                                   metrics =['CapMrktCurUSD', 'CapRealUSD', 'TxTfrValAdjUSD', 'PriceUSD',
                            'AdrActCnt', 'CapMVRVCur', 'NVTAdj','VelCur1yr', 'TxCnt', 'SplyCur', 'HashRate'] ,
                             start_time = '2018-01-01',
                             end_time = '2022-01-01',
                             frequency = '1d')


metrics = pd.DataFrame(metrics)
metrics.head()
metrics['time'] = pd.to_datetime(metrics['time'])
#df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
metrics['time'] = metrics['time'].dt.strftime('%Y-%m-%d')
#metrics.to_csv('metrics.csv')
#metrics.info()
#metrics.T

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.0 MB/s eta 0:00:00


In [ ]:
#metrics.describe() #count 1462 (because time range shorter than data : 5224)
#metrics.dtypes
#metrics[metrics.select_dtypes(include='object').columns] = metrics.select_dtypes(include='object').apply(pd.to_numeric, errors='coerce')
#print(metrics)
#metrics.keys()
#retrieve column
# Apply pd.to_numeric to all columns except 'time'
#metrics.loc[:, metrics.columns != 'time'] = metrics.loc[:, metrics.columns != 'time'].apply(pd.to_numeric, errors='coerce')
# Exclude the 'time' column, apply pd.to_numeric with errors='coerce' to force non-numeric entries to NaN
# and ensure the type change is applied back to the DataFrame.
cols_to_convert = metrics.columns.difference(['time'])  # Select all columns except 'time'
metrics[cols_to_convert] = metrics[cols_to_convert].apply(lambda x: pd.to_numeric(x, errors='coerce'))

metrics.dtypes

,0
asset,float64
time,object
AdrActCnt,int64
CapMVRVCur,float64
CapMrktCurUSD,float64
CapRealUSD,float64
HashRate,float64
NVTAdj,float64
PriceUSD,float64
SplyCur,float64


In [ ]:
'''selected_addresses = data[:int(len(data)*0.05)]
print(selected_addresses)
print("we take 5% of the total wallets by balance and limit to a total of", len(selected_addresses), "whales addresses")'''

'selected_addresses = data[:int(len(data)*0.05)]\nprint(selected_addresses)\nprint("we take 5% of the total wallets by balance and limit to a total of", len(selected_addresses), "whales addresses")'

# Column descriptions
**CapMrktCurUSD**: "The sum USD value of the current supply. Also referred to as network value or market capitalization."

**CapRealUSD:** "The sum USD value based on the USD closing price on the day that a native unit last moved (i.e., last transacted) for all native units."

**TxTfrValAdjUSD**: "The USD value of the sum of native units transferred between distinct addresses that interval removing noise and certain artifacts."

**PriceUSD**: "The fixed closing price of the asset as of 00:00 UTC the following day (i.e., midnight UTC of the current day) for end-of-day data or the closest prior hour (nearest to that block) for block-by-block data, denominated in USD."

In [ ]:
df = metrics

In [ ]:
# Helper function to calculate rolling metrics
def calculate_rolling_metric(df, metric_name, numerator, denominator, windows):
    series_names = []
    for window in windows:
        column_name = f"{metric_name}Roll{window}"
        df[column_name] = df[numerator] / df[denominator].rolling(window).mean()
        series_names.append(column_name)
    return series_names

In [ ]:
# Main function to calculate all metrics
def calculate_metrics(df):
    # NVT Signal Ratio (NVTS)
    df['NVTRatioAdj'] = df['CapMrktCurUSD'] / df['TxTfrValAdjUSD']

    # Metrics to calculate with configuration
    metrics_config = [
        {
            'name': 'NVTRatioAdjRoll',
            'numerator': 'CapMrktCurUSD',
            'denominator': 'TxTfrValAdjUSD',
            'windows': [7, 14, 28, 56, 70]
        },
        {
            'name': 'NVTSRatioRoll',
            'numerator': 'CapMrktCurUSD',
            'denominator': 'TxTfrValAdjUSD',
            'windows': [7, 14, 28, 56, 70, 90, 140]
        },
        {
            'name': 'RVTRatioRoll',
            'numerator': 'CapRealUSD',
            'denominator': 'TxTfrValAdjUSD',
            'windows': [7, 14, 28, 56, 70, 90, 140]
        },
    ]

    # Calculate each metric series based on configuration
    for config in metrics_config:
        calculate_rolling_metric(df, config['name'], config['numerator'], config['denominator'], config['windows'])

    # MVRV Ratio
    df['MVRV'] = df['CapMrktCurUSD'] / df['CapRealUSD']

    # Mayer Multiple
    df['MayerMultiple'] = df['PriceUSD'] / df['PriceUSD'].rolling(200).mean()

    # Mayer Multiple Standard Deviation
    df['MayerMultipleStDev'] = df['MayerMultiple'].rolling(200).std()

    return df

In [ ]:
# Run calculations
data = calculate_metrics(df)
#print(data)

In [ ]:
data.tail() # 35 col et ncp de NAn en 2009 et 2010 (range : 2009 -> 11/2024)

,asset,time,AdrActCnt,CapMVRVCur,CapMrktCurUSD,CapRealUSD,HashRate,NVTAdj,PriceUSD,SplyCur,...,RVTRatioRollRoll7,RVTRatioRollRoll14,RVTRatioRollRoll28,RVTRatioRollRoll56,RVTRatioRollRoll70,RVTRatioRollRoll90,RVTRatioRollRoll140,MVRV,MayerMultiple,MayerMultipleStDev
1457,NaN,2021-12-28,1051572,1.945825,9.016635e+11,4.633838e+11,1.616823e+08,61.282526,47673.044045,1.891349e+07,...,37.095193,36.021717,29.545010,26.129504,26.298087,25.789320,29.802176,1.945825,0.999109,0.207645
1458,NaN,2021-12-29,1142870,1.898460,8.789580e+11,4.629847e+11,1.749547e+08,44.559417,46470.321575,1.891439e+07,...,33.937300,35.010145,30.269768,26.101104,26.301865,25.821713,29.628604,1.898460,0.972802,0.207198
1459,NaN,2021-12-30,977727,1.923609,8.909496e+11,4.631656e+11,1.435835e+08,49.797208,47102.463004,1.891514e+07,...,32.237880,34.673172,30.852928,26.102134,26.409610,25.899495,29.495195,1.923609,0.985192,0.207037
1460,NaN,2021-12-31,1093396,1.893386,8.768566e+11,4.631157e+11,1.797810e+08,52.748215,46355.117330,1.891607e+07,...,31.572984,34.959872,31.471796,26.136611,26.397993,25.832777,29.377074,1.893386,0.968963,0.206999
1461,NaN,2022-01-01,695722,1.941966,8.996995e+11,4.632931e+11,2.075325e+08,192.234535,47560.009382,1.891714e+07,...,33.032520,35.708130,33.589173,26.268750,26.496446,26.010646,29.404494,1.941966,0.993390,0.206901


In [ ]:
import plotly.graph_objects as go

def plot_multiple_metrics(df, x_series='time', y1_series=None, y2_series=None,
                          title='Metrics Comparison',
                          y1_thresholds=None, y2_thresholds=None,
                          y1_axis_title='Metric (Y1)', y2_axis_title='Metric (Y2)',
                          y1_axis_type='linear', y2_axis_type='log'):
    """
    Plot multiple metrics on a dual-axis chart with optional thresholds and customized styling.

    Parameters:
        df: DataFrame containing the data.
        x_series: Column name in `df` for the x-axis.
        y1_series: List of dictionaries for left Y-axis series. Each dictionary should contain:
                   {'column': str, 'name': str, 'color': str}.
        y2_series: List of dictionaries for right Y-axis series. Each dictionary should contain:
                   {'column': str, 'name': str, 'color': str}.
        title: Plot title.
        y1_thresholds: Tuple of (lower, upper) thresholds for Y1 metrics. Applies color bands.
        y2_thresholds: Tuple of (lower, upper) thresholds for Y2 metrics. Applies color bands.
        y1_axis_title: Label for the left Y-axis.
        y2_axis_title: Label for the right Y-axis.
        y1_axis_type: Type for the left Y-axis ('linear' or 'log').
        y2_axis_type: Type for the right Y-axis ('linear' or 'log').
    """

    fig = go.Figure()

    # Plot each series on the left Y-axis
    if y1_series:
        for series in y1_series:
            fig.add_trace(go.Scatter(
                x=df[x_series], y=df[series['column']],
                mode='lines', name=series['name'],
                line=dict(color=series['color'], width=2.5),
                yaxis='y1'
            ))

    # Plot each series on the right Y-axis
    if y2_series:
        for series in y2_series:
            fig.add_trace(go.Scatter(
                x=df[x_series], y=df[series['column']],
                mode='lines', name=series['name'],
                line=dict(color=series['color'], width=2.5, dash='dash'),
                yaxis='y2'
            ))

    # Add threshold zones for Y1-axis
    if y1_thresholds:
        fig.add_hrect(
            y0=y1_thresholds[1], y1=df[y1_series[0]['column']].max(),
            fillcolor="rgba(255, 99, 71, 0.2)", line_width=0, yref="y1",
            annotation_text="Sell Zone", annotation_position="top left",
            annotation=dict(font_size=12, font_color="rgba(255, 99, 71, 0.8)")
        )
        fig.add_hrect(
            y0=df[y1_series[0]['column']].min(), y1=y1_thresholds[0],
            fillcolor="rgba(34, 139, 34, 0.2)", line_width=0, yref="y1",
            annotation_text="Buy Zone", annotation_position="bottom left",
            annotation=dict(font_size=12, font_color="rgba(34, 139, 34, 0.8)")
        )

    # Customize layout
    fig.update_layout(
        title=title,
        xaxis=dict(
            title='Date', tickformat='%Y-%m',
            showgrid=True, gridcolor='rgba(220,220,220,0.5)', color='black'
        ),
        yaxis=dict(
            title=y1_axis_title, type=y1_axis_type, side='left',
            showgrid=True, gridcolor='rgba(200,200,200,0.5)', color='black',
        ),
        yaxis2=dict(
            title=y2_axis_title, type=y2_axis_type, side='right',
            overlaying='y', showgrid=False, color='black'
        ),
        plot_bgcolor='white',
        legend=dict(x=0.5, y=1.15, xanchor='center', orientation='h'),
        hovermode="x unified",
        font=dict(color='black')
    )

    # Show the figure
    fig.show()




In [ ]:
df = data

Ajouter plusieurs métriques sur un seul graph

In [ ]:
# Example Usage with Multiple Metrics
plot_multiple_metrics(
    df,
    x_series='time',
    y1_series=[
        {'column': 'NVTRatioAdj', 'name': 'NVT Ratio Adjusted', 'color': 'teal'},
        {'column': 'RVTRatioRollRoll7', 'name': 'RVT Ratio (7-day)', 'color': 'blue'}
    ],
    y2_series=[
        {'column': 'PriceUSD', 'name': 'Price USD', 'color': 'gray'}
    ],
    title='NVT and RVT Ratios with Price (Dual-Axis)',
    y1_thresholds=(0.8, 1.5),
    y1_axis_title='NVT/RVT Ratios',
    y2_axis_title='Price USD',
    y1_axis_type='linear',
    y2_axis_type='log'
)


Sortir un graphe par métrique

In [ ]:
import plotly.graph_objects as go

def plot_multiple_metrics(df, x_series='time', y1_series=None, y2_series=None,
                          title='Metrics Comparison',
                          y1_thresholds=None,
                          y1_axis_title='Metric (Y1)', y2_axis_title='Metric (Y2)',
                          y1_axis_type='linear', y2_axis_type='log'):
    """
    Plot multiple metrics on a dual-axis chart with optional thresholds and customized styling.

    Parameters:
        df: DataFrame containing the data.
        x_series: Column name in `df` for the x-axis.
        y1_series: List of dictionaries for left Y-axis series. Each dictionary should contain:
                   {'column': str, 'name': str, 'color': str}.
        y2_series: List of dictionaries for right Y-axis series. Each dictionary should contain:
                   {'column': str, 'name': str, 'color': str}.
        title: Plot title.
        y1_thresholds: Tuple of (lower, upper) thresholds for Y1 metrics. Applies color bands.
        y1_axis_title: Label for the left Y-axis.
        y2_axis_title: Label for the right Y-axis.
        y1_axis_type: Type for the left Y-axis ('linear' or 'log').
        y2_axis_type: Type for the right Y-axis ('linear' or 'log').
    """

    fig = go.Figure()

    # Plot each series on the left Y-axis
    if y1_series:
        for series in y1_series:
            fig.add_trace(go.Scatter(
                x=df[x_series], y=df[series['column']],
                mode='lines', name=series['name'],
                line=dict(color=series['color'], width=2.5),
                yaxis='y1'
            ))

    # Plot each series on the right Y-axis
    if y2_series:
        for series in y2_series:
            fig.add_trace(go.Scatter(
                x=df[x_series], y=df[series['column']],
                mode='lines', name=series['name'],
                line=dict(color=series['color'], width=2.5, dash='dash'),
                yaxis='y2'
            ))

    # Add threshold zones for Y1-axis
    if y1_thresholds:
        fig.add_hrect(
            y0=y1_thresholds[1], y1=df[y1_series[0]['column']].max(),
            fillcolor="rgba(255, 99, 71, 0.2)", line_width=0, yref="y1",
            annotation_text="Sell Zone", annotation_position="top left",
            annotation=dict(font_size=12, font_color="rgba(255, 99, 71, 0.8)")
        )
        fig.add_hrect(
            y0=df[y1_series[0]['column']].min(), y1=y1_thresholds[0],
            fillcolor="rgba(34, 139, 34, 0.2)", line_width=0, yref="y1",
            annotation_text="Buy Zone", annotation_position="bottom left",
            annotation=dict(font_size=12, font_color="rgba(34, 139, 34, 0.8)")
        )

    # Customize layout
    fig.update_layout(
        title=title,
        xaxis=dict(
            title='Date', tickformat='%Y-%m',
            showgrid=True, gridcolor='rgba(220,220,220,0.5)', color='black'
        ),
        yaxis=dict(
            title=y1_axis_title, type=y1_axis_type, side='left',
            showgrid=True, gridcolor='rgba(200,200,200,0.5)', color='black',
        ),
        yaxis2=dict(
            title=y2_axis_title, type=y2_axis_type, side='right',
            overlaying='y', showgrid=False, color='black'
        ),
        plot_bgcolor='white',
        legend=dict(x=0.5, y=1.15, xanchor='center', orientation='h'),
        hovermode="x unified",
        font=dict(color='black')
    )

    # Show the figure
    fig.show()




Attention apparemment manque RVTRatioRoll dans le dataframe ,d'autre part pourquoi doublon RollRoll systématiquement dans le titre eg :  NVTSRatioRollRoll90

In [ ]:
def plot_metrics_series(df):
    """
    Iterates through multiple metric pairs and plots each one against 'PriceUSD'.
    """
    # Define metric pairs for plotting
    metric_pairs = [
        {'y1_metric': 'NVTRatioAdjRollRoll70', 'y1_title': 'NVT Ratio Adj Roll (70-day)'},
        {'y1_metric': 'NVTSRatioRollRoll90', 'y1_title': 'NVT Signal Ratio (90-day)'},
        {'y1_metric': 'RVTRatioRollRoll7', 'y1_title': 'RVT Ratio (7-day)'},
        {'y1_metric': 'MVRV', 'y1_title': 'MVRV Ratio'},
        {'y1_metric': 'MayerMultiple', 'y1_title': 'Mayer Multiple'},
        {'y1_metric': 'MayerMultipleStDev', 'y1_title': 'Mayer Multiple Std Dev'}
    ]

    # Loop through each metric pair and plot
    for pair in metric_pairs:
        plot_multiple_metrics(
            df,
            x_series='time',
            y1_series=[{'column': pair['y1_metric'], 'name': pair['y1_title'], 'color': 'teal'}],
            y2_series=[{'column': 'PriceUSD', 'name': 'Price USD', 'color': 'gray'}],
            title=f"{pair['y1_title']} vs Price USD",
            y1_thresholds=(0.8, 1.5),
            y1_axis_title=pair['y1_title'],
            y2_axis_title='Price USD',
            y1_axis_type='linear',
            y2_axis_type='log'
        )


# Execute the function to generate all plots
plot_metrics_series(df)


Voir tous les graphes dansune seule fenêtre pour faire side by side comparaison

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_metrics_grid(df):
    """
    Creates a grid of subplots for each metric against 'PriceUSD'.
    """
    # Define metric pairs for plotting with colors
    metric_pairs = [
        {'y1_metric': 'NVTRatioAdjRollRoll70', 'y1_title': 'NVT Ratio Adj Roll (70-day)'},
        {'y1_metric': 'NVTSRatioRollRoll90', 'y1_title': 'NVT Signal Ratio (90-day)'},
        {'y1_metric': 'RVTRatioRollRoll7', 'y1_title': 'RVT Ratio (7-day)'},
        {'y1_metric': 'MVRV', 'y1_title': 'MVRV Ratio'},
        {'y1_metric': 'MayerMultiple', 'y1_title': 'Mayer Multiple'},
        {'y1_metric': 'MayerMultipleStDev', 'y1_title': 'Mayer Multiple Std Dev'}
    ]

    # Define rows and columns for a 2x3 grid
    rows = 3
    cols = 2

    # Create subplots with shared x-axis and specify row/column titles
    fig = make_subplots(
        rows=rows, cols=cols,
        subplot_titles=[pair['y1_title'] for pair in metric_pairs],
        shared_xaxes=True,
        vertical_spacing=0.1,
        horizontal_spacing=0.1,
        specs=[[{"secondary_y": True}] * cols] * rows  # Enable secondary Y-axis for each subplot
    )

    # Loop through each metric pair and add it to the appropriate subplot
    for idx, pair in enumerate(metric_pairs):
        row = (idx // cols) + 1
        col = (idx % cols) + 1

        # Default color if 'color' key is missing
        line_color = pair.get('color', 'black')  # Fallback to black if color not found

        # Plot the metric on the left Y-axis
        fig.add_trace(go.Scatter(
            x=df['time'], y=df[pair['y1_metric']],
            mode='lines', name=pair['y1_title'],
            line=dict(color=line_color, width=2.5),
        ), row=row, col=col, secondary_y=False)

        # Plot PriceUSD on the right Y-axis for each subplot
        fig.add_trace(go.Scatter(
            x=df['time'], y=df['PriceUSD'],
            mode='lines', name='Price USD',
            line=dict(color='gray', width=2.5, dash='dash'),
        ), row=row, col=col, secondary_y=True)

        # Add thresholds for sell and buy zones (if applicable)
        if pair['y1_metric'] in ['NVTRatioAdjRollRoll70', 'NVTSRatioRollRoll90', 'RVTRatioRollRoll7']:
            fig.add_hrect(
                y0=1.5, y1=df[pair['y1_metric']].max(),
                fillcolor="rgba(255, 99, 71, 0.2)", line_width=0,
                row=row, col=col, secondary_y=False
            )
            fig.add_hrect(
                y0=df[pair['y1_metric']].min(), y1=0.8,
                fillcolor="rgba(34, 139, 34, 0.2)", line_width=0,
                row=row, col=col, secondary_y=False
            )

    # Update layout for the entire grid figure
    fig.update_layout(
        title="Metrics Comparison Grid",
        xaxis=dict(
            title='Date', tickformat='%Y-%m',
            showgrid=True, gridcolor='rgba(220,220,220,0.5)', color='black'
        ),
        plot_bgcolor='white',
        font=dict(color='black'),
        height=1200,
        showlegend=True,
        legend=dict(x=0.5, y=-0.15, xanchor='center', orientation='h')
    )

    # Customize individual y-axes
    for i, pair in enumerate(metric_pairs, start=1):
        fig['layout'][f'yaxis{i}'].update(title=pair['y1_title'], color=line_color)
        fig['layout'][f'yaxis{i*2}'].update(title="Price USD", color="gray", type="log")

    # Show the entire grid
    fig.show()

# Run the grid plot function
plot_metrics_grid(df)


In [ ]:
#files

In [ ]:
import requests
import json

def get_json_files(repo_url, headers=None):
    json_files = []

    # Recursive function to explore directories and find .json files
    def explore_directory(url):
        response = requests.get(url, headers=headers)
        items = response.json()

        for item in items:
            if item['type'] == 'file' and item['name'].endswith('.json'):
                # Add .json files to the list
                json_files.append(item['download_url'])
            elif item['type'] == 'dir':
                # Recursively explore directories
                explore_directory(item['url'])

    # Start with the main directory
    explore_directory(repo_url)
    return json_files

# Base URL for the Bitcoin directory in the GitHub API
bitcoin_repo_url = "https://api.github.com/repos/dataalways/CoinMetrics-formula-builder-models/contents/Bitcoin"
headers = {'Accept': 'application/vnd.github.v3+json'}  # GitHub API headers

# Get all JSON files
json_file_urls = get_json_files(bitcoin_repo_url, headers)

# Loop through JSON URLs to fetch and process formulas
for url in json_file_urls:
    response = requests.get(url, headers=headers)
    file_data = response.json()

    for metric in file_data.get("metrics", []):
        print("Metric Name:", metric.get("name"))
        print("Formula:", metric.get("formula"))


In [ ]:
#file_data.get()

In [ ]:
import requests
import json

def get_json_files(repo_url, headers=None):
    json_files = []

    # Recursive function to explore directories and find .json files
    def explore_directory(url):
        response = requests.get(url, headers=headers)

        # Ensure the response is JSON format
        try:
            items = response.json()  # Parse JSON
        except json.JSONDecodeError:
            print(f"Failed to decode JSON from {url}")
            return

        # If items is not a list, the response isn't in the expected format
        if not isinstance(items, list):
            print(f"Unexpected response format at {url}")
            return

        for item in items:
            if item['type'] == 'file' and item['name'].endswith('.json'):
                # Add .json files to the list
                json_files.append(item['download_url'])
            elif item['type'] == 'dir':
                # Recursively explore directories
                explore_directory(item['url'])

    # Start with the main directory
    explore_directory(repo_url)
    return json_files

# Base URL for the Bitcoin directory in the GitHub API
bitcoin_repo_url = "https://api.github.com/repos/dataalways/CoinMetrics-formula-builder-models/contents/Bitcoin"
headers = {'Accept': 'application/vnd.github.v3+json'}  # GitHub API headers

# Get all JSON files
json_file_urls = get_json_files(bitcoin_repo_url, headers)

# Loop through JSON URLs to fetch and process formulas
metrics_bis = []                                      #here  #metrics
for url in json_file_urls:
    response = requests.get(url, headers=headers)
    try:
        file_data = response.json()
        initial_data = file_data["content"]["extra"]["chartOptions"]["data"]["initialData"]
        for metric in initial_data:
            metric_name = metric.get("name")
            formula = metric.get("code")
            print(f"Metric Name: {metric_name}")
            print(f"Formula: {formula}\n")
            metrics_bis.append({"name": metric_name, "formula": formula})
#metrics.append({"name": metric["Metric Name"], "formula": metric["Formula"]})             #here
    except (KeyError, json.JSONDecodeError) as e:
        print(f"Skipping file {url} due to missing data structure or JSON decode error: {e}")




Metric Name: BTCUSD
Formula: BTC.PriceUSD

Metric Name: Buy Zone
Formula: sma(BTCUSD, 730)

Metric Name: Sell Zone
Formula: sma(BTCUSD, 730)*5

Metric Name: (BTC 4Y-CAGR + 1) [LHS] 
Formula: pow(BTC.PriceUSD/BTC.PriceUSD.shift(1460), 0.25).slice(BTC.PriceUSD.start+1460, BTC.PriceUSD.end)

Metric Name: BTC / USD [RHS]
Formula: BTC.PriceUSD

Metric Name: 1 ==  No Appreciation [LHS]
Formula: BTC.PriceBTC

Metric Name: BTC/ETH
Formula: rollingPearsonCorrelation(BTC.ReferenceRateNewYork0400pm, ETH.ReferenceRateNewYork0400pm, 30)

Metric Name: BTC/ADA
Formula: rollingPearsonCorrelation(BTC.ReferenceRateNewYork0400pm, ADA.ReferenceRateNewYork0400pm, 30)

Metric Name: BTC/SOL
Formula: rollingPearsonCorrelation(BTC.ReferenceRateNewYork0400pm, CMBISOL.ReferenceRateNewYork0400pm, 30)

Metric Name: BTC/XRP
Formula: rollingPearsonCorrelation(BTC.ReferenceRateNewYork0400pm, XRP.ReferenceRateNewYork0400pm, 30)

Metric Name: BTC/DOGE
Formula: rollingPearsonCorrelation(BTC.ReferenceRateNewYork0400pm, D

In [ ]:
metrics_bis

[{'name': 'BTCUSD', 'formula': 'BTC.PriceUSD'},
 {'name': 'Buy Zone', 'formula': 'sma(BTCUSD, 730)'},
 {'name': 'Sell Zone', 'formula': 'sma(BTCUSD, 730)*5'},
 {'name': '(BTC 4Y-CAGR + 1) [LHS] ',
  'formula': 'pow(BTC.PriceUSD/BTC.PriceUSD.shift(1460), 0.25).slice(BTC.PriceUSD.start+1460, BTC.PriceUSD.end)'},
 {'name': 'BTC / USD [RHS]', 'formula': 'BTC.PriceUSD'},
 {'name': '1 ==  No Appreciation [LHS]', 'formula': 'BTC.PriceBTC'},
 {'name': 'BTC/ETH',
  'formula': 'rollingPearsonCorrelation(BTC.ReferenceRateNewYork0400pm, ETH.ReferenceRateNewYork0400pm, 30)'},
 {'name': 'BTC/ADA',
  'formula': 'rollingPearsonCorrelation(BTC.ReferenceRateNewYork0400pm, ADA.ReferenceRateNewYork0400pm, 30)'},
 {'name': 'BTC/SOL',
  'formula': 'rollingPearsonCorrelation(BTC.ReferenceRateNewYork0400pm, CMBISOL.ReferenceRateNewYork0400pm, 30)'},
 {'name': 'BTC/XRP',
  'formula': 'rollingPearsonCorrelation(BTC.ReferenceRateNewYork0400pm, XRP.ReferenceRateNewYork0400pm, 30)'},
 {'name': 'BTC/DOGE',
  'formu

In [ ]:
import pandas as pd
import numpy as np
import requests

# Define your data and helper functions
data3 = pd.DataFrame(df)  # your existing DataFrame with all columns     here the names is data 2
def sma(series, window): return series.rolling(window=window).mean()
def rollingPearsonCorrelation(series1, series2, window): return series1.rolling(window).corr(series2)

#Step 2 : def get_json_files (cf plus haut)

# Step 3: Dynamically calculate each metric
metric_results = {}
for metric in metrics_bis :               #metrics
    try:
        metric_name = metric["name"]
        metric_formula = metric["formula"]
        metric_results[metric_name] = eval(metric_formula)  # Evaluate formula in current context
    except Exception as e:
        print(f"Error calculating {metric_name}: {e}")

# Step 4: Convert results to a DataFrame for easier use and display
metric_df = pd.DataFrame(metric_results)
print(metric_df.head())


Error calculating BTCUSD: name 'BTC' is not defined
Error calculating Buy Zone: name 'BTCUSD' is not defined
Error calculating Sell Zone: name 'BTCUSD' is not defined
Error calculating (BTC 4Y-CAGR + 1) [LHS] : name 'BTC' is not defined
Error calculating BTC / USD [RHS]: name 'BTC' is not defined
Error calculating 1 ==  No Appreciation [LHS]: name 'BTC' is not defined
Error calculating BTC/ETH: name 'BTC' is not defined
Error calculating BTC/ADA: name 'BTC' is not defined
Error calculating BTC/SOL: name 'BTC' is not defined
Error calculating BTC/XRP: name 'BTC' is not defined
Error calculating BTC/DOGE: name 'BTC' is not defined
Error calculating BTC/DOT: name 'BTC' is not defined
Error calculating BTC/LINK: name 'BTC' is not defined
Error calculating BTC/SPX 30D : name 'BTC' is not defined
Error calculating BTC/ETH 30D : name 'BTC' is not defined
Error calculating Daily %MC Transaction Fees [120d SMA]: name 'BTC' is not defined
Error calculating Daily %MC Block Rewards [120d SMA]: nam

In [ ]:
metrics.T


,0,1,2,3,4,5,6,7,8,9,...,1452,1453,1454,1455,1456,1457,1458,1459,1460,1461
asset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
time,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08,2018-01-09,2018-01-10,...,2021-12-23,2021-12-24,2021-12-25,2021-12-26,2021-12-27,2021-12-28,2021-12-29,2021-12-30,2021-12-31,2022-01-01
AdrActCnt,972783,1142721,1130917,1253986,1069525,1163503,1130388,1157303,1145338,1118949,...,991240,974212,866151,774902,992531,1051572,1142870,977727,1093396,695722
CapMVRVCur,2.694235,2.925443,2.960267,2.944022,3.261964,3.24693,3.042781,2.801525,2.690171,2.733345,...,2.070151,2.071635,2.064058,2.071542,2.070718,1.945825,1.89846,1.923609,1.893386,1.941966
CapMrktCurUSD,225887722580.948517,247555189825.181396,251879335164.348572,252918055361.86145,285287177348.308899,287108308287.372375,272505179860.495331,250807207092.34375,241453646852.438721,246362365863.323151,...,958932834186.89917,960320724015.785278,956710327599.571045,960683606726.740967,960694114316.437134,901663524613.240112,878957961885.919678,890949566257.024292,876856588254.334229,899699536194.449341
CapRealUSD,83841120873.070969,84621433103.613297,85086698666.29985,85909025021.172852,87458707327.674881,88424557641.229919,89557927689.441406,89525228754.022995,89754021493.672028,90132198891.487869,...,463218883753.959106,463556990766.975037,463509477789.73407,463752827367.302063,463942433153.012146,463383765865.054504,462984715971.554565,463165554253.850342,463115694273.528625,463293135888.826782
HashRate,14923655.718422,16415540.667357,15071578.273538,16127548.725824,13727615.879719,16127548.725824,15551564.842759,15935554.098136,15743559.470447,15935554.098136,...,182818645.184793,181615889.983017,177168715.849691,161682279.298922,178574457.266352,161682279.298922,174954703.603701,143583516.198115,179781040.662247,207532476.142665
NVTAdj,49.915092,41.630404,44.707399,34.291802,32.287149,39.388679,33.617113,36.48311,36.568967,35.829978,...,74.810301,66.153339,104.224008,115.026472,59.332345,61.282526,44.559417,49.797208,52.748215,192.234535
PriceUSD,13464.653612,14754.322205,15010.28616,15070.300799,16997.227408,17103.58928,16231.694999,14937.415089,14378.586217,14669.088266,...,50712.970768,50783.834421,50590.450898,50798.306586,50796.377674,47673.044045,46470.321575,47102.463004,46355.11733,47560.009382
SplyCur,16776348.586185,16778486.086185,16780448.586185,16782548.586185,16784336.086184,16786436.086184,16788461.086184,16790536.086124,16792586.086104,16794661.086094,...,18909025.041518,18909968.791502,18910887.541502,18911725.041502,18912650.041502,18913487.541464,18914393.791431,18915137.541431,18916068.791431,18917143.791431


In [ ]:
#df_cleaned_apply = df[df.apply(lambda row: pd.notna(row[["CapMrktCurUSD", "MayerMultipleStDev", "NVTRatioAdj", "Price Change"]]).all(), axis=1)]
metrics["Price Change"] = metrics["PriceUSD"].pct_change().fillna(0)
df_cleaned_apply = metrics[metrics.apply(lambda row: pd.notna(row[["CapMrktCurUSD", "MayerMultipleStDev", "NVTRatioAdj", "Price Change"]]).all(), axis=1)]

In [ ]:
df_cleaned_apply.T

,398,399,400,401,402,403,404,405,406,407,...,1452,1453,1454,1455,1456,1457,1458,1459,1460,1461
asset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
time,2019-02-03,2019-02-04,2019-02-05,2019-02-06,2019-02-07,2019-02-08,2019-02-09,2019-02-10,2019-02-11,2019-02-12,...,2021-12-23,2021-12-24,2021-12-25,2021-12-26,2021-12-27,2021-12-28,2021-12-29,2021-12-30,2021-12-31,2022-01-01
AdrActCnt,518121,610696,645414,686774,645404,691966,561955,501124,709903,809914,...,991240,974212,866151,774902,992531,1051572,1142870,977727,1093396,695722
CapMVRVCur,0.773435,0.773847,0.777657,0.765109,0.763789,0.821601,0.823679,0.828481,0.817172,0.817076,...,2.070151,2.071635,2.064058,2.071542,2.070718,1.945825,1.89846,1.923609,1.893386,1.941966
CapMrktCurUSD,59788219917.066002,59765870504.425873,60036309266.967644,59010177291.223839,58866207389.19751,63318160367.386528,63478991740.723839,63849176538.515076,62954510736.379906,62912631290.854248,...,958932834186.89917,960320724015.785278,956710327599.571045,960683606726.740967,960694114316.437134,901663524613.240112,878957961885.919678,890949566257.024292,876856588254.334229,899699536194.449341
CapRealUSD,77302162952.08374,77232112153.010117,77201492506.530701,77126451928.02681,77071331151.969772,77066772569.847809,77067636793.24144,77067724659.45903,77039524206.783585,76997326554.561295,...,463218883753.959106,463556990766.975037,463509477789.73407,463752827367.302063,463942433153.012146,463383765865.054504,462984715971.554565,463165554253.850342,463115694273.528625,463293135888.826782
HashRate,39888680.944089,42490116.657834,42490116.657834,47403939.672685,50294423.799069,47114891.260047,43357261.895749,41080343.910222,43691361.641039,38568926.138297,...,182818645.184793,181615889.983017,177168715.849691,161682279.298922,178574457.266352,161682279.298922,174954703.603701,143583516.198115,179781040.662247,207532476.142665
NVTAdj,89.153684,73.198626,70.705206,61.525478,65.262072,56.853105,84.24643,99.682983,63.04301,56.388679,...,74.810301,66.153339,104.224008,115.026472,59.332345,61.282526,44.559417,49.797208,52.748215,192.234535
PriceUSD,3413.021616,3411.387962,3426.465011,3367.506365,3358.873599,3612.479619,3621.268121,3642.022359,3590.618484,3587.902466,...,50712.970768,50783.834421,50590.450898,50798.306586,50796.377674,47673.044045,46470.321575,47102.463004,46355.11733,47560.009382
SplyCur,17517679.826995,17519517.326995,17521354.826995,17523404.826995,17525579.826995,17527617.32699,17529492.32699,17531242.32699,17533054.82699,17534654.826984,...,18909025.041518,18909968.791502,18910887.541502,18911725.041502,18912650.041502,18913487.541464,18914393.791431,18915137.541431,18916068.791431,18917143.791431


In [ ]:


import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import mean_squared_error

# Drop rows where 'A' or 'B' have NaN values using apply
df_cleaned_apply = df[df.apply(lambda row: pd.notna(row[["CapMrktCurUSD", "MayerMultipleStDev", "NVTRatioAdj", "Price Change"]]).all(), axis=1)]
df = df_cleaned_apply
print("DataFrame after dropping rows with NaN in specified columns using apply:\n", df_cleaned_apply)

df['Date'] = df['time']
df["Price Change"] = df["PriceUSD"].pct_change().fillna(0)
#df["ETH_Stablecoin_Ratio"] = df["balance"].apply(float) / (df["balance_total_stablecoins"] + 1e-9)    #ETH balance

# Select and normalize features
features = df[["CapMrktCurUSD", "MayerMultipleStDev", "NVTRatioAdj", "Price Change"]]
#features = df[["balance", "balance_total_stablecoins", "ETH_Stablecoin_Ratio", "Price Change","Date"]]
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)
target = df["PriceUSD"].values.reshape(-1, 1)
scaled_target = scaler.fit_transform(target)

# Split data into train, validation, and test sets (60%, 20%, 20%)
train_size = int(0.6 * len(scaled_features))
val_size = int(0.2 * len(scaled_features))

X_train, X_val, X_test = scaled_features[:train_size], scaled_features[train_size:train_size+val_size], scaled_features[train_size+val_size:]
y_train, y_val, y_test = scaled_target[:train_size], scaled_target[train_size:train_size+val_size], scaled_target[train_size+val_size:]

# Model definition
model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(32, activation="relu"),
    Dense(1)
])

# Compile model
model.compile(optimizer="adam", loss="mse")

# Early stopping based on validation loss, with patience of 10 epochs
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Training the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=80,
    callbacks=[early_stopping],
    batch_size=16,
    verbose=1
)

# Sequential predictions on the test set
test_predictions = model.predict(X_test)
test_predictions = scaler.inverse_transform(test_predictions).flatten()

# Performance threshold filtering
threshold = 1.5  # 1.5% sensitivity control
filtered_predictions = [
    pred if abs(pred - true) / true * 100 <= threshold else None
    for pred, true in zip(test_predictions, scaler.inverse_transform(y_test).flatten())
]

# Plotting - Actual vs Predicted Prices with on-chain metrics
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Plot actual prices
fig.add_trace(
    go.Scatter(
        x=df["Date"],
        y=df["PriceUSD"],
        mode="lines",
        name="Actual Price (USD)",
        line=dict(color="blue", width=2),
    ),
    secondary_y=False,
)

# Plot predicted prices over test set period
test_dates = df["Date"][train_size+val_size:]
fig.add_trace(
    go.Scatter(
        x=test_dates,
        #y=test_predictions,
        y=filtered_predictions,
        mode="lines",
        name="Predicted Price (USD)",
        line=dict(color="orange", dash="dash"),
    ),
    secondary_y=False,
)

# Plot ETH Balance and Stablecoin Balance
fig.add_trace(
    go.Scatter(
        x=df["Date"],
        y=df["CapMrktCurUSD"],  #CapMrktCurUSD not ETH Balance
        mode="lines",
        name="CapMrktCurUSD",
        line=dict(color="green", width=1),
    ),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(
        x=df["Date"],
        y=df["MayerMultipleStDev"],  #  MayerMultipleStDev not "Stablecoin Balance",
        mode="lines",
        name="MayerMultipleStDev",
        line=dict(color="purple", width=1),
    ),
    secondary_y=True,
)

# Update layout
fig.update_layout(
    title="Ethereum Price Forecast with On-Chain Metrics",
    xaxis_title="Date",
    yaxis_title="Price (USD)",
    yaxis2_title="On-Chain Metrics",
    legend=dict(orientation="h", yanchor="top", y=1.15, xanchor="center", x=0.5),
    font=dict(family="Arial", size=12),
    plot_bgcolor="white"
)

# Set y-axes to log scale for clarity
fig.update_yaxes(type="log", secondary_y=False)
fig.update_yaxes(type="log", secondary_y=True)

fig.show()

# Print final model validation performance
val_mse = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation MSE: {val_mse}")

# Print final model test performance with threshold
test_mse = mean_squared_error(scaler.inverse_transform(y_test), test_predictions)
print(f"Test MSE with threshold: {test_mse}")




DataFrame after dropping rows with NaN in specified columns using apply:
       asset        time  AdrActCnt  CapMVRVCur  CapMrktCurUSD    CapRealUSD  \
398     NaN  2019-02-03     518121    0.773435   5.978822e+10  7.730216e+10   
399     NaN  2019-02-04     610696    0.773847   5.976587e+10  7.723211e+10   
400     NaN  2019-02-05     645414    0.777657   6.003631e+10  7.720149e+10   
401     NaN  2019-02-06     686774    0.765109   5.901018e+10  7.712645e+10   
402     NaN  2019-02-07     645404    0.763789   5.886621e+10  7.707133e+10   
...     ...         ...        ...         ...            ...           ...   
1457    NaN  2021-12-28    1051572    1.945825   9.016635e+11  4.633838e+11   
1458    NaN  2021-12-29    1142870    1.898460   8.789580e+11  4.629847e+11   
1459    NaN  2021-12-30     977727    1.923609   8.909496e+11  4.631656e+11   
1460    NaN  2021-12-31    1093396    1.893386   8.768566e+11  4.631157e+11   
1461    NaN  2022-01-01     695722    1.941966   8.996995

<ipython-input-28-83ffb4cddf55>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-28-83ffb4cddf55>:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.0045 - val_loss: 0.2197
Epoch 2/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 7.7710e-04 - val_loss: 0.1458
Epoch 3/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.5029e-04 - val_loss: 0.0948
Epoch 4/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 4.7751e-05 - val_loss: 0.0701
Epoch 5/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.4998e-05 - val_loss: 0.0641
Epoch 6/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2.3476e-05 - val_loss: 0.0607
Epoch 7/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.7582e-05 - val_loss: 0.0596
Epoch 8/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.2035e-05 - val_loss: 0.0560
Epoch 9/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2507e-05 - val_loss: 0.0554
Epoch 10/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0604e-05 - val_loss: 0.0514
Epoch 11/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.4212e-06 - val_loss: 0.0497
Epoch 12/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3m

Validation MSE: 0.025593208149075508
Test MSE with threshold: 159404365.38396692


In [ ]:


import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import mean_squared_error

'''# Drop rows where 'A' or 'B' have NaN values using apply
df_cleaned_apply = df[df.apply(lambda row: pd.notna(row[["CapMrktCurUSD", "MayerMultipleStDev", "NVTRatioAdj", "Price Change"]]).all(), axis=1)]
df = df_cleaned_apply
print("DataFrame after dropping rows with NaN in specified columns using apply:\n", df_cleaned_apply)

df['Date'] = df['time']
df["Price Change"] = df["PriceUSD"].pct_change().fillna(0)
#df["ETH_Stablecoin_Ratio"] = df["balance"].apply(float) / (df["balance_total_stablecoins"] + 1e-9)    #ETH balance'''

# Select and normalize features
features = df[["CapMrktCurUSD", "MayerMultipleStDev", "NVTRatioAdj", "Price Change"]]


# Select and normalize features
features = df[["CapMrktCurUSD", "MayerMultipleStDev", "NVTRatioAdj", "Price Change"]]
#features = df[["balance", "balance_total_stablecoins", "ETH_Stablecoin_Ratio", "Price Change"]]
#features = df[["balance", "balance_total_stablecoins", "ETH_Stablecoin_Ratio", "Price Change","Date"]]
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)
target = df["PriceUSD"].values.reshape(-1, 1)
scaled_target = scaler.fit_transform(target)

# Split data into train, validation, and test sets (60%, 20%, 20%)
train_size = int(0.6 * len(scaled_features))
val_size = int(0.2 * len(scaled_features))

X_train, X_val, X_test = scaled_features[:train_size], scaled_features[train_size:train_size+val_size], scaled_features[train_size+val_size:]
y_train, y_val, y_test = scaled_target[:train_size], scaled_target[train_size:train_size+val_size], scaled_target[train_size+val_size:]

# Model definition
model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    Dense(32, activation="relu"),
    Dense(1)
])

# Compile model
model.compile(optimizer="adam", loss="mse")

# Early stopping based on validation loss, with patience of 10 epochs
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Training the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=80,
    callbacks=[early_stopping],
    batch_size=16,
    verbose=1
)

# Sequential predictions on the test set
test_predictions = model.predict(X_test)
test_predictions = scaler.inverse_transform(test_predictions).flatten()

# Performance threshold filtering
threshold = 1.5  # 1.5% sensitivity control
filtered_predictions = [
    pred if abs(pred - true) / true * 100 <= threshold else None
    for pred, true in zip(test_predictions, scaler.inverse_transform(y_test).flatten())
]

# Plotting - Actual vs Predicted Prices with on-chain metrics
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Plot actual prices
fig.add_trace(
    go.Scatter(
        x=df["Date"],
        y=df["PriceUSD"],
        mode="lines",
        name="Actual Price (USD)",
        line=dict(color="blue", width=2),
    ),
    secondary_y=False,
)

# Plot predicted prices over test set period
test_dates = df["Date"][train_size+val_size:]
fig.add_trace(
    go.Scatter(
        x=test_dates,
        #y=test_predictions,
        y=filtered_predictions,
        mode="lines",
        name="Predicted Price (USD)",
        line=dict(color="orange", dash="dash"),
    ),
    secondary_y=False,
)


# Plot ETH Balance and Stablecoin Balance
fig.add_trace(
    go.Scatter(
        x=df["Date"],
        y=df["CapMrktCurUSD"],  #CapMrktCurUSD not ETH Balance
        mode="lines",
        name="CapMrktCurUSD",
        line=dict(color="green", width=1),
    ),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(
        x=df["Date"],
        y=df["MayerMultipleStDev"],  #  MayerMultipleStDev not "Stablecoin Balance",
        mode="lines",
        name="MayerMultipleStDev",
        line=dict(color="purple", width=1),
    ),
    secondary_y=True,
)

# Update layout
fig.update_layout(
    title="Ethereum Price Forecast with On-Chain Metrics",
    xaxis_title="Date",
    yaxis_title="Price (USD)",
    yaxis2_title="On-Chain Metrics",
    legend=dict(orientation="h", yanchor="top", y=1.15, xanchor="center", x=0.5),
    font=dict(family="Arial", size=12),
    plot_bgcolor="white"
)

# Set y-axes to log scale for clarity
fig.update_yaxes(type="log", secondary_y=False)
fig.update_yaxes(type="log", secondary_y=True)

fig.show()

# Print final model validation performance
val_mse = model.evaluate(X_val, y_val, verbose=0)
print(f"Validation MSE: {val_mse}")

# Print final model test performance with threshold
test_mse = mean_squared_error(scaler.inverse_transform(y_test), test_predictions)
print(f"Test MSE with threshold: {test_mse}")




Epoch 1/80


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0270 - val_loss: 0.3254
Epoch 2/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0011 - val_loss: 0.2818
Epoch 3/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.2764e-04 - val_loss: 0.2608
Epoch 4/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.0608e-04 - val_loss: 0.2246
Epoch 5/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.4497e-04 - val_loss: 0.2063
Epoch 6/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.1035e-04 - val_loss: 0.1820
Epoch 7/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.3823e-04 - val_loss: 0.1620
Epoch 8/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.8853e-04 - val_loss: 0.1421
Epoch 9/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.5873e-04 - val_loss: 0.1247
Epoch 10/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.7322e-04 - val_loss: 0.1107
Epoch 11/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.4426e-04 - val_loss: 0.0959
Epoch 12/80
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/ste

Validation MSE: 0.036603499203920364
Test MSE with threshold: 225193632.40393317


In [ ]:
'''eth_price.csv 'date' 'open_price'
df['momentum_returns']  = df['open_price'].pct_changes(periods=n)
df['signal'] = 0 # 0  = holds, 1 = buy , -1  sell
df.loc[df['momentum_returns']  > 0, 'signal'] = 1 #buy signal
df.loc[df['momentum_returns']  < 0, 'signal'] = -1 #sell signal
##############
compare our model to this simple momentum strategy '''


"eth_price.csv 'date' 'open_price' \ndf['momentum_returns']  = df['open_price'].pct_changes(periods=n)\ndf['signal'] = 0 # 0  = holds, 1 = buy , -1  sell \ndf.loc[df['momentum_returns']  > 0, 'signal'] = 1 #buy signal \ndf.loc[df['momentum_returns']  < 0, 'signal'] = -1 #sell signal \n##############\ncompare our model to this simple momentum strategy "

In [ ]:


import unittest
import pandas as pd

# Assume the calculate_metrics function and helper functions are already defined
# from previous code, and imported here.

class TestMetricsCalculations(unittest.TestCase):

    def setUp(self):
        # Define a small sample dataset to use in tests
        '''self.data = pd.DataFrame({
            'CapMrktCurUSD': [100, 150, 200, 250, 300],
            'TxTfrValAdjUSD': [10, 15, 20, 25, 30],
            'CapRealUSD': [80, 120, 160, 200, 240],
            'PriceUSD': [1, 2, 3, 4, 5]
        })'''
        self.data = data                        #Fonctionne avec le vrai dataset mais pas avec le jeu de test (revoir peut être le jeu de test)
        # Apply the main calculation function
        self.result = calculate_metrics(self.data.copy())

    def test_nvtratioadj_calculation(self):
        # Check if 'NVTRatioAdj' is calculated correctly
        expected_nvtratioadj = self.data['CapMrktCurUSD'] / self.data['TxTfrValAdjUSD']
        pd.testing.assert_series_equal(self.result['NVTRatioAdj'], expected_nvtratioadj, check_names=False)

    def test_rolling_nvt_ratios(self):
        # Check if rolling NVT Ratios for each window size are calculated correctly
        for window in [7, 14, 28, 56, 70]:
            column_name = f'NVTRatioAdjRoll{window}'
            expected = self.data['NVTRatioAdj'].rolling(window).mean()
            pd.testing.assert_series_equal(self.result[column_name], expected, check_names=False)

    def test_nvts_signal_ratio(self):
        # Check if NVTS signal ratios are calculated correctly for each window size
        for window in [7, 14, 28, 56, 70, 90, 140]:
            column_name = f'NVTSRatioRoll{window}'
            expected = self.data['CapMrktCurUSD'] / self.data['TxTfrValAdjUSD'].rolling(window).mean()
            pd.testing.assert_series_equal(self.result[column_name], expected, check_names=False)

    def test_rvtratio_calculation(self):
        # Check if RVT Ratios are calculated correctly for each window size
        for window in [7, 14, 28, 56, 70, 90, 140]:
            column_name = f'RVTRatioRoll{window}'
            expected = self.data['CapRealUSD'] / self.data['TxTfrValAdjUSD'].rolling(window).mean()
            pd.testing.assert_series_equal(self.result[column_name], expected, check_names=False)

    def test_mvrv_ratio(self):
        # Check if MVRV Ratio is calculated correctly
        expected_mvrv = self.data['CapMrktCurUSD'] / self.data['CapRealUSD']
        pd.testing.assert_series_equal(self.result['MVRV'], expected_mvrv, check_names=False)

    def test_mayer_multiple(self):
        # Check if Mayer Multiple is calculated correctly
        column_name = 'MayerMultiple'
        expected_mayer_multiple = self.data['PriceUSD'] / self.data['PriceUSD'].rolling(200).mean()
        pd.testing.assert_series_equal(self.result[column_name], expected_mayer_multiple, check_names=False)

    def test_mayer_multiple_stdev(self):
        # Check if Mayer Multiple standard deviation is calculated correctly
        column_name = 'MayerMultipleStDev'
        expected_mayer_stdev = self.result['MayerMultiple'].rolling(200).std()
        pd.testing.assert_series_equal(self.result[column_name], expected_mayer_stdev, check_names=False)

    def test_empty_dataframe(self):
        # Check if function can handle an empty DataFrame without errors
        empty_df = pd.DataFrame(columns=self.data.columns)
        result = calculate_metrics(empty_df)
        self.assertTrue(result.empty)

    def test_single_row_dataframe(self):
        # Check function with a single row, expecting NaNs for rolling windows
        single_row_df = pd.DataFrame({
            'CapMrktCurUSD': [100],
            'TxTfrValAdjUSD': [10],
            'CapRealUSD': [80],
            'PriceUSD': [1]
        })
        result = calculate_metrics(single_row_df)
        # All rolling window columns should be NaN
        for column in result.columns:
            if 'Roll' in column:
                self.assertTrue(result[column].isna().all())

'''# Run the tests
if __name__ == '__main__':
    unittest.main()'''

# Run the tests in Google Colab
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


.....EEE.
ERROR: test_nvts_signal_ratio (__main__.TestMetricsCalculations)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'NVTSRatioRoll7'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "<ipython-input-31-1b3cdbf4d32a>", line 38, in test_nvts_signal_ratio
    pd.testing.assert_series_equal(self.result[column_name], expected, check_names=False)
  F